In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import hydra
import torch
import xarray as xr

from crims2s.training.infer import find_checkpoint_file
from crims2s.training.lightning import S2STercilesModule

In [ ]:
with hydra.initialize_config_module('crims2s.training.conf'):
    cfg = hydra.compose('config', overrides=['experiment=emos'])

In [ ]:
checkpoint_path = find_checkpoint_file(
    hydra.utils.to_absolute_path('***HOME***fast_scratch/s2s/runs/train/outputs/2021-09-18/13-56-18')
)

checkpoint_path

In [ ]:
chkpt = torch.load(checkpoint_path, map_location=torch.device('cpu'))

In [ ]:
state_dict = chkpt['state_dict']

In [ ]:
state_dict

In [ ]:
model_state = {k[6:]: state_dict[k] for k in state_dict if k.startswith('model.')}

In [ ]:
model = hydra.utils.instantiate(cfg.experiment.model)
optimizer = hydra.utils.call(cfg.experiment.optimizer, model)

lightning_module = S2STercilesModule.load_from_checkpoint(
    checkpoint_path, model=model, optimizer=optimizer
)

In [ ]:
lightning_module.model

In [ ]:
loc_model = model.model.tp_model.loc_model

In [ ]:
scale_model = model.model.tp_model.scale_model

In [ ]:
model_weights = []
for k in sorted(loc_model.models):
    model_weights.append(scale_model.models[k].weights)

In [ ]:
params = torch.stack(model_weights)

In [ ]:
params.shape

In [ ]:
params.max()

In [ ]:
params.mean()

In [ ]:
params.std()

In [ ]:
params.shape

In [ ]:
params[:, 0].mean(dim=0)

In [ ]:
params_array = xr.DataArray(params.detach(), dims=['week', 'lead_time', 'latitude', 'longitude'])
params_array = params_array.reindex(latitude=params_array.latitude[::-1])

In [ ]:
params_array.isel(lead_time=0).mean(dim='week').plot()